In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os

from shapely.geometry import LineString

# remote i/o

In [2]:
# project directory
root_dir = os.path.join("D:/metcouncil_network_rebuild")

external_dir = os.path.join(root_dir, "data", "external")
interim_dir = os.path.join(root_dir, "data", "interim")

In [3]:
routing_result_dir = interim_dir

In [6]:
gtfs_input_dir = os.path.join(external_dir, 'gtfs')

gtfs_feed_names = ['MT2019', 'MV2019']

# read gtfs and routing results

In [37]:
routing_df = gpd.read_file(os.path.join(routing_result_dir, 'bus_routing.geojson'))
routing_df.rename(columns = {'source':'routing_method'}, inplace = True)

stop_times_df = pd.DataFrame()
stops_df = pd.DataFrame()
routes_df = pd.DataFrame()
trips_df = pd.DataFrame()
shapes_df = pd.DataFrame()

for feed in gtfs_feed_names:
    temp_stop_times_df = pd.read_csv(
        os.path.join(gtfs_input_dir, feed, 'stop_times.txt'), dtype = {'trip_id':object, 'stop_id':object}
    )
    temp_stop_times_df['agency_raw_name'] = feed
    
    stop_times_df = stop_times_df.append(temp_stop_times_df, sort = False, ignore_index = True)

    temp_stops_df = pd.read_csv(
        os.path.join(gtfs_input_dir, feed, 'stops.txt'), dtype = {'stop_id':object}
    )
    temp_stops_df['agency_raw_name'] = feed
    
    stops_df = stops_df.append(temp_stops_df, sort = False, ignore_index = True)

    temp_routes_df = pd.read_csv(
        os.path.join(gtfs_input_dir, feed, 'routes.txt')
    )
    temp_routes_df['agency_raw_name'] = feed
    
    routes_df = routes_df.append(temp_routes_df, sort = False, ignore_index = True)

    temp_trips_df = pd.read_csv(
        os.path.join(gtfs_input_dir, feed, 'trips.txt'), dtype = {'trip_id':object, 'shape_id':object}
    )
    temp_trips_df['agency_raw_name'] = feed
    
    trips_df = trips_df.append(temp_trips_df, sort = False, ignore_index = True)

    temp_shapes_df = pd.read_csv(
        os.path.join(gtfs_input_dir, feed, 'shapes.txt'), dtype = {'shape_id':object}
    )
    temp_shapes_df['agency_raw_name'] = feed
    
    shapes_df = shapes_df.append(temp_shapes_df, sort = False, ignore_index = True)

In [38]:
stops_df

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,wheelchair_boarding,agency_raw_name,parent_station
0,1000,NaN,50th St W & Upton Ave S,Near side E,44.912365,-93.315179,NaN,http://www.metrotransit.org/NexTripBadge.aspx?...,0,1.0,MT2019,NaN
1,10000,NaN,Carmen Ave & Claude Way E,Across from S,44.857531,-93.041008,NaN,http://www.metrotransit.org/NexTripBadge.aspx?...,0,1.0,MT2019,NaN
2,10001,NaN,Carmen Ave & 65th St E,Far side S,44.855001,-93.042491,NaN,http://www.metrotransit.org/NexTripBadge.aspx?...,0,1.0,MT2019,NaN
3,10002,NaN,Carmen Ave & Village Square Center,Near side S,44.853643,-93.042243,NaN,http://www.metrotransit.org/NexTripBadge.aspx?...,0,1.0,MT2019,NaN
4,10003,NaN,Carmen Ave & Lake Bridge Apts,Across from E,44.853399,-93.040785,NaN,http://www.metrotransit.org/NexTripBadge.aspx?...,0,2.0,MT2019,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13844,place_BTST,NaN,Burnsville Transit Station,NaN,44.778813,-93.276049,NaN,NaN,1,NaN,MV2019,NaN
13845,place_CE47,NaN,Cedar & 147th (BRT),NaN,44.735146,-93.217624,NaN,NaN,1,NaN,MV2019,NaN
13846,place_DFME,NaN,Diffley Rd @ Metcalf JHS,NaN,44.804841,-93.226214,NaN,NaN,1,NaN,MV2019,NaN
13847,place_EATS,NaN,Eagan Transit Station,NaN,44.831925,-93.166537,NaN,NaN,1,NaN,MV2019,NaN


In [39]:
routing_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 223534 entries, 0 to 223533
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   u                   223534 non-null  int64   
 1   v                   223534 non-null  int64   
 2   agency_raw_name     223534 non-null  object  
 3   shape_id            223534 non-null  object  
 4   trip_id             223534 non-null  object  
 5   fromIntersectionId  223534 non-null  object  
 6   toIntersectionId    223534 non-null  object  
 7   shstReferenceId     223534 non-null  object  
 8   shstGeometryId      223534 non-null  object  
 9   agency_shape_id     223534 non-null  object  
 10  method              223534 non-null  object  
 11  geometry            223534 non-null  geometry
dtypes: geometry(1), int64(2), object(9)
memory usage: 20.5+ MB


# add route name to trips

In [40]:
# add route names to trips

trips_df = pd.merge(
    trips_df,
    routes_df[['agency_raw_name','route_id', 'route_short_name', 'route_long_name']],
    how = 'left',
    on = ['agency_raw_name','route_id']
)

In [41]:
trips_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52053 entries, 0 to 52052
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   route_id               52053 non-null  object 
 1   service_id             52053 non-null  object 
 2   trip_id                52053 non-null  object 
 3   trip_headsign          52053 non-null  object 
 4   direction_id           52053 non-null  int64  
 5   block_id               52049 non-null  float64
 6   shape_id               52053 non-null  object 
 7   wheelchair_accessible  49731 non-null  float64
 8   agency_raw_name        52053 non-null  object 
 9   trip_destination       2322 non-null   object 
 10  trip_short_name        2322 non-null   object 
 11  trip_route_name        2322 non-null   object 
 12  pub_dir_id             2322 non-null   object 
 13  rm_block_id            2318 non-null   object 
 14  route_short_name       52053 non-null  object 
 15  ro

# add trip info to ranch routing result

In [42]:
# add trip info to routing result

routing_df = pd.merge(
    routing_df,
    trips_df[['agency_raw_name', 'trip_id', 'shape_id', 'route_id', 'direction_id', 'route_short_name', 'route_long_name']],
    how='left',
    on =['agency_raw_name', 'trip_id', 'shape_id']
)

routing_df['source'] = 'routing'

In [43]:
routing_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 223534 entries, 0 to 223533
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   u                   223534 non-null  int64   
 1   v                   223534 non-null  int64   
 2   agency_raw_name     223534 non-null  object  
 3   shape_id            223534 non-null  object  
 4   trip_id             223534 non-null  object  
 5   fromIntersectionId  223534 non-null  object  
 6   toIntersectionId    223534 non-null  object  
 7   shstReferenceId     223534 non-null  object  
 8   shstGeometryId      223534 non-null  object  
 9   agency_shape_id     223534 non-null  object  
 10  method              223534 non-null  object  
 11  geometry            223534 non-null  geometry
 12  route_id            223534 non-null  object  
 13  direction_id        223534 non-null  int64   
 14  route_short_name    223534 non-null  object  
 15  route_lon

In [44]:
trips_df = trips_df[trips_df.trip_id.isin(routing_df.trip_id.unique())]

In [45]:
trips_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 865 entries, 2 to 51798
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   route_id               865 non-null    object 
 1   service_id             865 non-null    object 
 2   trip_id                865 non-null    object 
 3   trip_headsign          865 non-null    object 
 4   direction_id           865 non-null    int64  
 5   block_id               865 non-null    float64
 6   shape_id               865 non-null    object 
 7   wheelchair_accessible  625 non-null    float64
 8   agency_raw_name        865 non-null    object 
 9   trip_destination       240 non-null    object 
 10  trip_short_name        240 non-null    object 
 11  trip_route_name        240 non-null    object 
 12  pub_dir_id             240 non-null    object 
 13  rm_block_id            240 non-null    object 
 14  route_short_name       865 non-null    object 
 15  rout

# create stop time shapes from gtfs

In [50]:
stop_times_line_gdf

,geometry
trip_id,
--411.0N-1721-20190518-Weekday-04,"LINESTRING (-93.47152 44.73090, -93.50657 44.7..."
--411.0N-1721-20190518-Weekend-01,"LINESTRING (-93.47152 44.73090, -93.50657 44.7..."
--411.0N-1721-20190914-Weekday-03,"LINESTRING (-93.47152 44.73090, -93.50657 44.7..."
--411.0N-1721-20190914-Weekend-02,"LINESTRING (-93.47152 44.73090, -93.50657 44.7..."
001-S1-420.3E-0705-20190518-Weekday-04,"LINESTRING (-93.21791 44.72575, -93.12380 44.7..."
...,...
952--411.0S-2134-20190914-Weekend-02,"LINESTRING (-93.50657 44.77599, -93.47152 44.7..."
952--411.0S-2326-20190518-Weekday-04,"LINESTRING (-93.50657 44.77599, -93.47152 44.7..."
952--411.0S-2326-20190518-Weekend-01,"LINESTRING (-93.50657 44.77599, -93.47152 44.7..."


In [51]:
stop_times_df = pd.merge(
    stop_times_df,
    stops_df[['agency_raw_name', 'stop_id', 'stop_lat', 'stop_lon']],
    how='left',
    on =['agency_raw_name', 'stop_id']
)

stop_times_gdf = gpd.GeoDataFrame(
    stop_times_df,
    geometry = gpd.points_from_xy(stop_times_df['stop_lon'], stop_times_df['stop_lat']),
    crs = routing_df.crs
)

stop_times_line_gdf = stop_times_gdf.sort_values(by=['stop_sequence']).groupby(
    ['agency_raw_name', 'trip_id']
)['geometry'].apply(
    lambda x: LineString(x.tolist())
)

stop_times_line_gdf = gpd.GeoDataFrame(stop_times_line_gdf, geometry = 'geometry')

stop_times_line_gdf = pd.merge(
    stop_times_line_gdf,
    trips_df[['agency_raw_name', 'trip_id', 'shape_id', 'route_id', 'direction_id', 'route_short_name', 'route_long_name']],
    how='inner',
    on =['agency_raw_name', 'trip_id']
)

stop_times_line_gdf['source'] = 'stop_times'

shapes_df = pd.merge(
    shapes_df,
    trips_df[['trip_id', 'shape_id', 'route_id', 'direction_id', 'route_short_name', 'route_long_name']],
    how = 'inner',
    on = ['shape_id']
)

# create shapes from gtfs

In [52]:
shapes_gdf = gpd.GeoDataFrame(
    shapes_df,
    geometry = gpd.points_from_xy(shapes_df['shape_pt_lon'], shapes_df['shape_pt_lat']),
    crs = routing_df.crs
)

shapes_line_gdf = shapes_gdf.sort_values(by=['shape_pt_sequence']).groupby(['agency_raw_name' ,'shape_id'])['geometry'].apply(
    lambda x: LineString(x.tolist())
)

shapes_line_gdf = gpd.GeoDataFrame(shapes_line_gdf, geometry = 'geometry')

shapes_line_gdf = pd.merge(
    shapes_line_gdf,
    trips_df[['agency_raw_name', 'trip_id', 'shape_id', 'route_id', 'direction_id', 'route_short_name', 'route_long_name']],
    how = 'inner',
    on = ['agency_raw_name', 'shape_id']
)

shapes_line_gdf['source'] ='shapes'

# combining the 3 data

In [53]:
print(routing_df.columns)
print(stop_times_line_gdf.columns)
print(shapes_line_gdf.columns)

Index(['u', 'v', 'agency_raw_name', 'shape_id', 'trip_id',
       'fromIntersectionId', 'toIntersectionId', 'shstReferenceId',
       'shstGeometryId', 'agency_shape_id', 'method', 'geometry', 'route_id',
       'direction_id', 'route_short_name', 'route_long_name', 'source'],
      dtype='object')
Index(['agency_raw_name', 'trip_id', 'geometry', 'shape_id', 'route_id',
       'direction_id', 'route_short_name', 'route_long_name', 'source'],
      dtype='object')
Index(['agency_raw_name', 'shape_id', 'geometry', 'trip_id', 'route_id',
       'direction_id', 'route_short_name', 'route_long_name', 'source'],
      dtype='object')


In [54]:
routing_df.trip_id.nunique()

865

In [55]:
stop_times_line_gdf.trip_id.nunique()

865

In [56]:
shapes_line_gdf.trip_id.nunique()

865

In [57]:
shapes_line_gdf.shape_id.nunique()

578

In [58]:
routing_df.shape_id.nunique()

578

In [59]:
out_df = pd.concat(
    [routing_df, stop_times_line_gdf, shapes_line_gdf],
    sort = False,
    ignore_index = True
)

# write out tableau data

In [60]:
out_df.to_file(os.path.join(routing_result_dir, 'routing-reivew.geojson'), driver = 'GeoJSON')